<a href="https://colab.research.google.com/github/CarlosMendez1997Col/GeoDatabases-And-Cloud-Computing-For-Water-Resources-Management/blob/main/1-Creation%20Geodatabase/Download_and_Geoprocessing_Databases_in_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and Geoprocessing Databases in Google Colab

---

> Water Resources Management using PostgreSQL and PgAdmin4

> Area of Interest (South America)

> Developed by MSc Carlos Mendez

MOST TABLES AND DATASETS USED:

1. South America Countries and Boundary
2. Firt Level Administrative Units (FLAU)
3. Second Level Administrative Units (SLAU)
4. HydroSHEDS (Conditioned DEM)
5. HydroBASINS (Level 1,2,3,4,5,6,7,8,9,10,11 and 12)
6. HydroRIVERS
7. HydroLAKES
8. Global Lakes and Wetlands Database (GLWD)
9. HydroWASTE
10. Global River Classification (GloRiC)
11. Lake TEMP
12. Global Power Plant Database (GPPD)



## Install and import ArcGIS API for Python

In [1]:
!pip install arcgis

## Import libraries and packages

In [5]:
import numpy as np
import pandas as pd
import math
import os
import time
from datetime import datetime as dt

In [3]:
import arcgis

from google.colab import output
output.enable_custom_widget_manager()

# connect to GIS
from arcgis.gis import GIS

## Connect and Login in Arcgis Account

In [4]:
# Prompt user to provide username and password
import getpass
username = input('Enter username: ')
password = getpass.getpass("Enter your password: ")
gis = GIS("https://udistritalfjc.maps.arcgis.com/home", username, password)

Enter username: camendezv_UDFJC
Enter your password: ··········


## Import and extract Databases in your local computer

1. South America Countries and Boundary [Url Data](https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM0.zip)

2. Firt Level Administrative Units (FLAU) [Url Data](https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.zip)

3. Second Level Administrative Units (SLAU) [Url Data](https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM2.zip)

4. HydroSHEDS (Digital Elevation Model) [Url Data](https://data.hydrosheds.org/file/hydrosheds-v1-dem/hyd_sa_dem_30s.zip)

5. HydroBASINS (Level 1,2,3,4,5,6,7,8,9,10,11 and 12) [Url Data](https://data.hydrosheds.org/file/hydrobasins/standard/hybas_sa_lev01-12_v1c.zip)

6. HydroRIVERS [Url Data](https://data.hydrosheds.org/file/HydroRIVERS/HydroRIVERS_v10_sa_shp.zip)

7. HydroLAKES [Url Data](https://data.hydrosheds.org/file/hydrolakes/HydroLAKES_polys_v10_shp.zip)

8. Global Lakes and Wetlands Database (GLWD) [Url Data](https://figshare.com/ndownloader/files/54001748)

9. HydroWASTE [Url Data](https://figshare.com/ndownloader/files/31910714)

10. Global River Classification (GloRiC) [Url Data](https://data.hydrosheds.org/file/hydrosheds-associated/gloric/GloRiC_v10_shapefile.zip)

11. Lake TEMP [Url Data](https://figshare.com/ndownloader/files/46397785)

12. Global Power Plant Database (GPPD) [Url Data](https://datasets.wri.org/private-admin/dataset/53623dfd-3df6-4f15-a091-67457cdb571f/resource/66bcdacc-3d0e-46ad-9271-a5a76b1853d2/download/globalpowerplantdatabasev130.zip)

### Connect to Google Drive

In [27]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: Mountpoint must be in a directory that exists

In [24]:
!pwd

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
pwd: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [16]:
## 1. South America Countries and Boundary

!wget https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM0.zip

!unzip https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM0.zip

--2025-09-08 23:47:15--  https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM0.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/wmgeolab/geoBoundaries/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM0.zip [following]
--2025-09-08 23:47:16--  https://media.githubusercontent.com/media/wmgeolab/geoBoundaries/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM0.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103997008 (99M) [application/zip]
Saving to: ‘geoBoundariesCGAZ_ADM0.zip.2’

geoBoundariesCGAZ_A 100%[===================>]  99.18M  58.2MB/s    in

In [23]:
!ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [22]:
!rm -r /content/

In [ ]:



## 2. Firt Level Administrative Units (FLAU)
https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.zip

## 3. Second Level Administrative Units (SLAU)
https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM2.zip

## 4. HydroSHEDS (Digital Elevation Model)
https://data.hydrosheds.org/file/hydrosheds-v1-dem/hyd_sa_dem_30s.zip

## 5. HydroBASINS (Level 1,2,3,4,5,6,7,8,9,10,11 and 12)
https://data.hydrosheds.org/file/hydrobasins/standard/hybas_sa_lev01-12_v1c.zip

## 6. HydroRIVERS
https://data.hydrosheds.org/file/HydroRIVERS/HydroRIVERS_v10_sa_shp.zip

## 7. HydroLAKES
https://data.hydrosheds.org/file/hydrolakes/HydroLAKES_polys_v10_shp.zip

## 8. Global Lakes and Wetlands Database (GLWD)
https://figshare.com/ndownloader/files/54001748

##9. HydroWASTE
https://figshare.com/ndownloader/files/31910714

## 10. Global River Classification (GloRiC)
https://data.hydrosheds.org/file/hydrosheds-associated/gloric/GloRiC_v10_shapefile.zip

## 11. Lake TEMP
https://figshare.com/ndownloader/files/46397785

## 12. Global Power Plant Database (GPPD)
https://datasets.wri.org/private-admin/dataset/53623dfd-3df6-4f15-a091-67457cdb571f/resource/66bcdacc-3d0e-46ad-9271-a5a76b1853d2/download/globalpowerplantdatabasev130.zip


!wget your_external_url_to_zip_file.zip